### 1.4 决策树分类器（Decision Tree) / 集成分类器（Ensemble Tree）
之前的分类器大多有一下几点缺陷：   
* 线性分类器对于特征与类别直接的关系是“线性假设”，如果遇到非线性的关系，就很难辨识，比如Titanic数据中，如果假设“年龄”与“生存”是正相关的，那么年龄越大，越有可能生存；但是事实证明，这个假设是错误的，不是正相关，而偏偏是老人与小孩更加容易获得生存的机会。这种情况，线性假设不完全成立，因此，需要非线性的分类器。
* 即便使用类似SVM的分类器，我们很难得到明确分类“依据”的说明，无法“解释”分类器是如何工作的，特别无法从人类逻辑的角度理解。高维度、不可解释性等，这些都是弊端。
决策树分类器解决了上述两点问题。我们使用Titanic（泰坦尼克号的救援记录）这个数据集来实践一个预测某乘客是否获救的分类器。 【Source Code】 

In [1]:
import pandas as pd
import numpy as np

titanic = pd.read_csv('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic.txt')

In [2]:
#瞧瞧数据，什么数据特征的都有，有数值型的、类别型的，字符串，甚至还有缺失的数据等等。
titanic.head()

,row.names,pclass,survived,name,age,embarked,home.dest,room,ticket,boat,sex
0,1,1st,1,"Allen, Miss Elisabeth Walton",29.0000,Southampton,"St Louis, MO",B-5,24160 L221,2,female
1,2,1st,0,"Allison, Miss Helen Loraine",2.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
2,3,1st,0,"Allison, Mr Hudson Joshua Creighton",30.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,(135),male
3,4,1st,0,"Allison, Mrs Hudson J.C. (Bessie Waldo Daniels)",25.0000,Southampton,"Montreal, PQ / Chesterville, ON",C26,NaN,NaN,female
4,5,1st,1,"Allison, Master Hudson Trevor",0.9167,Southampton,"Montreal, PQ / Chesterville, ON",C22,NaN,11,male


In [3]:
# 使用pandas，数据都转入pandas独有的dataframe格式（二维数据表格），直接使用info()，查看数据的基本特征
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 11 columns):
row.names    1313 non-null int64
pclass       1313 non-null object
survived     1313 non-null int64
name         1313 non-null object
age          633 non-null float64
embarked     821 non-null object
home.dest    754 non-null object
room         77 non-null object
ticket       69 non-null object
boat         347 non-null object
sex          1313 non-null object
dtypes: float64(1), int64(2), object(8)
memory usage: 112.9+ KB


* 这份调查数据是真实的泰坦尼克号乘客个人和登船信息，有助于我们预测乘客是否幸免。
* 一共1313条数据 有些特征是完整的（比如 pclass，survived，name）； 有些是有缺失的；有些事数值类型的信息（age:float64）,有些是字符串。
* 机器学习有一个不太被初学者重视，并且耗时，你是十分重要的一环，** 特征的选择 **，这个需要基于一些背景知识。根据我们对这场事故的了解，sex，age，pclass这些都很有可能是决定幸免与否的关键因素

In [4]:
X = titanic[['pclass','age','sex']]# 选择多列
y = titanic['survived']# 选择一列

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       633 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


## 下面有几个对数据处理的任务
1. age 这个数据列，只有633个
2. sex 与 pclass 两个数据列的值都是类别型的，需要转化为数值特征，用0/1代替

## 首先我们补充age里的数据，使用平均数或者中位数都是对I模型偏离造成最小影响的策略


In [6]:
X['age'].fillna(X['age'].mean(),inplace = True) # 用某值替换  空 元素

C:\Users\97855\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [7]:
X.info() # 查看age是否完全

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1313 entries, 0 to 1312
Data columns (total 3 columns):
pclass    1313 non-null object
age       1313 non-null float64
sex       1313 non-null object
dtypes: float64(1), object(2)
memory usage: 30.9+ KB


In [8]:
from sklearn.cross_validation import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25,random_state = 33)

In [9]:
# 我们使用scikit-learn中的feature_extraction，进行特征提取（one-hot编码）
# 查看http://sklearn.lzjqsdd.com/modules/feature_extraction.html
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer(sparse = False)
X_train = vec.fit_transform(X_train.to_dict(orient = 'record'))
# X_train.to_dict(orient = 'record')->http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_dict.html?highlight=to_dict
print(vec.feature_names_)

['age', 'pclass=1st', 'pclass=2nd', 'pclass=3rd', 'sex=female', 'sex=male']


### 我们发现，凡是类别型的特征都单独剥离出来，读成一列特征，数值型特征则保持不变。

In [10]:
X_test = vec.transform(X_test.to_dict(orient = 'record'))

In [11]:
from sklearn.tree import DecisionTreeClassifier

# 决策树分类器，使用信息熵作为创建树好坏的衡量标准
# http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
dtc = DecisionTreeClassifier(criterion = 'entropy',max_depth = 3,min_samples_leaf = 5)
dtc.fit(X_train,y_train)
dtc.score(X_test,y_test)

0.79331306990881456

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(max_depth = 3,min_samples_leaf = 5)

gbc.fit(X_train,y_train)
gbc.score(X_test,y_test)

0.79027355623100304

In [13]:
from sklearn.metrics import classification_report

y_predict = gbc.predict(X_test)
print(classification_report(y_predict,y_test))

             precision    recall  f1-score   support

          0       0.93      0.78      0.84       241
          1       0.57      0.83      0.68        88

avg / total       0.83      0.79      0.80       329

